In [14]:
import math
import time
import wikionly #script name is wikionly (no summary), class name is wiki
import re as re
import nltk
# nltk.download('wordnet')
from nltk.corpus import wordnet
import math

#Input two Wikipedia articles to compute similarity percentage
class similar:
    def __init__(self,text1,text2,verbose=1):
        """To start, assign var = comparewiki.similar('arg1','arg2', verbose=1). 
        arg1 and arg2 are names of the wikipedia articles.
        verbose=1 prints the probability score and mathematical calculation. 
        verbose=2 additionally prints array of words for each article
        verbose=0 disables any logs.
        To get values in a list for storage, use .ans(). To get the 40 common words for comparison, use .words()"""
        time01 = time.time()
        self.wn = nltk.corpus.wordnet #the corpus reader
        self.verbose = verbose # Verbose/log level of detail
        time02 = time.time()
        print("Initialize NLTK similarity corpus reader: " + str(time02 - time01) + " seconds.")
        
        #Error handling: check if both arguments input are string format
        checkstr = False
        if isinstance(text1, str) == True:
            if isinstance(text2, str) == True:
                self.text1 = text1
                self.text2 = text2
                time03 = time.time()
                print("String check of arguments: " + str(time03 - time02) + " seconds.")
                checkstr = True
            else:
                print('Error! The second argument is not a string format!')        
        else:
            print('Error! The first argument is not a string format!')
        
        #Run internal wikipedia python file for processing for both wiki titles
        if checkstr == True:
            self.wiki1 = wikionly.wiki(text1)
            time04 = time.time()
            print("Scrape wikipedia for article 1: " + str(round(time04 - time03, 4)) + " seconds.")
            self.wiki2 = wikionly.wiki(text2)
            time05 = time.time()
            print("Scrape wikipedia for article 2: " + str(round(time05 - time04, 4)) + " seconds.")
            
        #Call the function that calculates percentage
        self.percent(self.wiki1,self.wiki2,self.verbose)
        
        #call the function that shows list of words for both Wiki sites
        #Only can be used if self.percent has been called and list/arrays for articles are created
        if self.verbose == 2:
            print(self.words())

    #Retrieve top 40 common words from wiki page, slice up and append .n01 for NLTK usage
    def percent(self,input1,input2,verbose):
        time06 = time.time()
        self.dotn01 = ('.','n','.','0','1')
        self.wiki1list = []
        for key in self.wiki1.commonwords(40):
            self.wiki1slice = list(key)
            for letter in self.dotn01:
                self.wiki1slice.append(letter)
            self.wiki1slice = ''.join(self.wiki1slice)
            self.wiki1list.append(self.wiki1slice)

        self.wiki2list = []
        for key in self.wiki2.commonwords(40):
            self.wiki2slice = list(key)
            for letter in self.dotn01:
                self.wiki2slice.append(letter)
            self.wiki2slice = ''.join(self.wiki2slice)
            self.wiki2list.append(self.wiki2slice)
            
        time07 = time.time()
        print("Get list of 40 words for both articles: " + str(round(time07 - time06, 4)) + " seconds.")
        
        #count and sum for calculating similarity
        self.count = 0
        self.sum = 0
        #A count for the ranking of the word (how often it appears in both wiki passages)
        self.topten1 = 0
        self.topten2 = 0

        #For words that are 1-10th and 11-21st in popularity, if both wiki pages have the word, they get more points
        for word1 in self.wiki1list:
            #Reset self.topten2
            self.topten2 = 0
            self.topten1 += 1
            for word2 in self.wiki2list:
                self.topten2 += 1
                #reinitialize to zero to prevent old sums from going into maxsum
                self.sum1 = 0
                self.sum2 = 0
                self.sum3 = 0
                self.sum4 = 0
                self.maxsum = 0
                
                if self.topten1 < 11 and self.topten2 < 11:
                    self.expvalue = 4.5
                elif self.topten1 < 21 and self.topten2 < 21:
                    self.expvalue = 2.5
                else:
                    self.expvalue = 1.5
                
                #Main algorithm for calculating score of words
                try:
                    if re.findall(r"\d+.n.01", word1) == [] and re.findall(r"\d+.n.01", word2) == []: #check both words not numbers
                        #since words have many meanings, for every pair of words, use top two meanings n.01 and n.02 for comparison
                        #two for loops will check every permutation pair of words between wiki pages, two meanings for each word, 
                        #Take the max similarity value taken for computation of similarity index
                        #e.g. money.n.01 may have highest value with value.n.02 because value.n.01 has the obvious meaning of worth/significance and secondary for money
                        word11 = word1.replace('n.01','n.02')
                        word22 = word2.replace('n.01','n.02')
                        #print(word11,word22)
                        self.x = self.wn.synset(word1)
                        self.y = self.wn.synset(word2)
                        #get default similarity value of 1st definitions of word
                        self.sum1 = self.x.path_similarity(self.y) * math.exp(self.expvalue * self.x.path_similarity(self.y)) + 10 * math.log(0.885+self.x.path_similarity(self.y))
                        try: #get 2nd definitions of words and their similarity values, if it exist
                            self.xx = self.wn.synset(word11)
                            self.yy = self.wn.synset(word22)
                            self.sum2 = self.xx.path_similarity(self.y) * math.exp(self.expvalue * self.xx.path_similarity(self.y)) + 10 * math.log(0.89+self.xx.path_similarity(self.y))
                            self.sum3 = self.x.path_similarity(self.yy) * math.exp(self.expvalue * self.x.path_similarity(self.yy)) + 10 * math.log(0.89+self.x.path_similarity(self.yy))
                            self.sum4 = self.xx.path_similarity(self.yy) * math.exp(self.expvalue * self.xx.path_similarity(self.yy)) + 10 * math.log(0.89+self.xx.path_similarity(self.yy))
                        except:
                            continue
                        self.maxsum = max(self.sum1,self.sum2,self.sum3,self.sum4) #get the max similarity value between 2 words x 2 meanings = 4 comparisons
                        #print(word1, word2, self.maxsum)
                        self.sum += self.maxsum
                        self.count += 1
                except:
                    if word1 == word2 and re.findall(r"\d+.n.01", word1) == []: #remove years/numbers being counted as match yyyy.n.01
                        self.sum += math.exp(self.expvalue) + 10 * math.log(1.89)
                        self.count += 1
                    else:
                        continue

        time08 = time.time()
        print("Calculate similarity for both articles: " + str(round(time08 - time07, 4)) + " seconds.")
        
        #Print the results and implement ceiling if the percent exceeds 100% or drops below 0%
        if self.count != 0:
            self.pct = round(self.sum/self.count*100)
            if self.pct > 100:
                self.pct = 100
            elif self.pct < 0:
                self.pct = 0
            if self.verbose >= 1:
                print('Probability of topics being related is ' + str(self.pct) + '%')
                print('Count is ' + str(self.count) + ' and sum is ' + str(self.sum))
        else:
            if self.verbose >= 1:
                print('No relation index can be calculated as words are all foreign')
                
        time09 = time.time()
        print("Print output: " + str(time09 - time08) + " seconds.")
        return self.pct
        
    #Print out list of common words for both Wiki articles
    def words(self):
        print(self.wiki1list)
        print('\n')
        print(self.wiki2list)
        
    #Outputs list of results [Article 1, Article 2, Percentage, Yes/No] that can be put into a dataframe
    def ans(self):
        self.listans = [self.text1,self.text2,self.pct]
        if self.pct > 49:
            self.listans.append('Yes')
        else:
            self.listans.append('No')
        
        if self.verbose == 2:
            self.listans.append(self.wiki1list)
            self.listans.append(self.wiki2list)
        
        return self.listans
    
    def help(self):
        print("To start, assign var = comparewiki.similar('arg1','arg2', verbose=1). arg1 and arg2 are names of the wikipedia articles, while verbose=1 prints the probability score and mathematical calculation. verbose=2 additionally prints array of words for each article, and verbose=0 disables any logs. To get values in a list for storage, use .ans(). To get the 40 common words for comparison, use .words()")

In [15]:
a = similar('Joe Biden','Donald Trump')

Initialize NLTK similarity corpus reader: 0.0 seconds.
String check of arguments: 0.0 seconds.
Scrape wikipedia for article 1: 3.8429 seconds.
Scrape wikipedia for article 2: 5.8538 seconds.
Get list of 40 words for both articles: 0.0036 seconds.
Calculate similarity for both articles: 1.428 seconds.
Probability of topics being related is 97%
Count is 407 and sum is 396.52122327972694
Print output: 0.0020904541015625 seconds.


In [16]:
b = similar('John F Kennedy','Donald Trump')

Initialize NLTK similarity corpus reader: 0.0 seconds.
String check of arguments: 0.0 seconds.
Scrape wikipedia for article 1: 4.5453 seconds.
Scrape wikipedia for article 2: 4.9795 seconds.
Get list of 40 words for both articles: 0.003 seconds.
Calculate similarity for both articles: 1.086 seconds.
Probability of topics being related is 77%
Count is 349 and sum is 268.68840002368
Print output: 0.0019996166229248047 seconds.


In [22]:
b = similar('Tony Blair','Donald Trump')

Initialize NLTK similarity corpus reader: 0.0 seconds.
String check of arguments: 0.0 seconds.
Scrape wikipedia for article 1: 2.5158 seconds.
Scrape wikipedia for article 2: 6.3029 seconds.
Get list of 40 words for both articles: 0.003 seconds.
Calculate similarity for both articles: 1.2261 seconds.
Probability of topics being related is 43%
Count is 385 and sum is 166.8504644575315
Print output: 0.002000570297241211 seconds.


In [18]:
x = similar('Tony Blair','HP')

Initialize NLTK similarity corpus reader: 0.0 seconds.
String check of arguments: 0.0029976367950439453 seconds.
Scrape wikipedia for article 1: 2.27 seconds.

The title "HP" you specified is ambiguous. As a result, you are linked to a clarification page.


Here are some suggestions to use: 

None
None
None
wiktionary:HP
wiktionary:hp
HP Inc.
Horsepower
None
None
None
None
None
None
Harry Pearson (audio critic)
Hit points
Horse-Power: Ballet Symphony
Hewlett-Packard
Hewlett Packard Enterprise
Penguin Group
HP Foods
HP Sauce
Handley Page
Hindustan Petroleum
America West Airlines
Heart Peaks
Himachal Pradesh
Hunters Point (San Francisco)
HP postcode area
Haptoglobin
Hypersensitivity pneumonitis
Ilford HP
High precipitation supercell
High pressure
Hollow point
Horizontal pitch
Security hacker
Phreaking
Half-pay
Haltepunkt
Hire purchase
Howiesons Poort
Hybrid perpetual
HP Garage
None
Scrape wikipedia for article 2: 0.1462 seconds.
Get list of 40 words for both articles: 0.002 seconds.
Calc

In [19]:
z = similar('NP','HP')

Initialize NLTK similarity corpus reader: 0.0 seconds.
String check of arguments: 0.0007612705230712891 seconds.

The title "NP" you specified is ambiguous. As a result, you are linked to a clarification page.


Here are some suggestions to use: 

None
None
None
None
None
None
None
None
None
None
None
None
None
NP (novel)
Banana Yoshimoto
National Party (disambiguation)
Ngee Ann Polytechnic
Nigeria Police Force
Northern Pacific Railway
November Project
NP postcode area
Nepal
.np
Nucleoside phosphorylase
Nurse practitioner
Kallikrein 8
Neptunium
NP (complexity)
NP-complete
NP-hard
Co-NP
Numpy
Named Pipe
P-n junction
Neper
Neptunium
Power number
International Organization for Standardization
Not populated
Not placed
Nanoparticle
Noun phrase
Notary public
Enpi (disambiguation)
None
Scrape wikipedia for article 1: 0.5552 seconds.

The title "HP" you specified is ambiguous. As a result, you are linked to a clarification page.


Here are some suggestions to use: 

None
None
None
wiktionary:H

In [20]:
trance = similar('Armin van Buuren','Tiesto')

Initialize NLTK similarity corpus reader: 0.0 seconds.
String check of arguments: 0.0 seconds.
Scrape wikipedia for article 1: 0.7086 seconds.
Scrape wikipedia for article 2: 0.9174 seconds.
Get list of 40 words for both articles: 0.002 seconds.
Calculate similarity for both articles: 1.6015 seconds.
Probability of topics being related is 95%
Count is 511 and sum is 486.3398758941804
Print output: 0.0017197132110595703 seconds.


In [21]:
country = similar('Singapore','Japan')

Initialize NLTK similarity corpus reader: 0.0 seconds.
String check of arguments: 0.0009982585906982422 seconds.
Scrape wikipedia for article 1: 2.9781 seconds.
Scrape wikipedia for article 2: 2.9265 seconds.
Get list of 40 words for both articles: 0.003 seconds.
Calculate similarity for both articles: 1.3053 seconds.
Probability of topics being related is 95%
Count is 408 and sum is 387.96742212073457
Print output: 0.002003192901611328 seconds.
